<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 安裝 gspread 和 pandas 函式庫
!pip install -q gspread pandas

import gradio as gr
import pandas as pd
from datetime import datetime
import gspread
from google.colab import auth
from google.auth import default

# 2. 您的 Google Sheet 資訊 (從您提供的連結中擷取)
# 試算表 ID
SPREADSHEET_ID = '1IoDPXfU7vTqloUcgFeEbd5_aoASIamXRFJ-WXq9ubtc'
# 工作表索引 (gid=0 通常是第一個工作表)
WORKSHEET_INDEX = 0

# ⚠️ 3. Google 帳號授權 (必須執行此步驟，點擊連結並完成授權)
print("正在進行 Google 授權...")
auth.authenticate_user()
# 取得授權憑證
creds, _ = default()
gc = gspread.authorize(creds)

# 4. 連線到您的試算表和工作表
try:
    spreadsheet = gc.open_by_key(SPREADSHEET_ID)
    worksheet = spreadsheet.get_worksheet(WORKSHEET_INDEX)
    print("✅ Google Sheets 連線成功！")
except gspread.exceptions.SpreadsheetNotFound:
    print(f"❌ 錯誤：找不到 ID 為 {SPREADSHEET_ID} 的試算表。請檢查 ID 是否正確。")
except Exception as e:
    # 這通常是因為權限不足。
    print(f"❌ 錯誤：連線或權限不足。請檢查您的 Google 試算表是否已開啟分享給您的 Google 帳號。\n詳細錯誤: {e}")

# 預先載入資料以供 Gradio 介面初始顯示 (如果連線成功)
if 'worksheet' in locals():
    try:
        data_records = worksheet.get_all_records()
        df_expenses = pd.DataFrame(data_records)
    except Exception as e:
        print(f"⚠️ 警告：讀取資料發生錯誤，可能試算表為空或欄位不正確。將使用空表格。錯誤: {e}")
        df_expenses = pd.DataFrame(columns=["Date", "Amount", "Category", "Description"])
else:
    # 建立空的 DataFrame 以避免 Gradio 啟動錯誤
    df_expenses = pd.DataFrame(columns=["Date", "Amount", "Category", "Description"])

# 設置 Dataframe，讓最新紀錄在最上面 (反轉順序)
initial_df_display = df_expenses.iloc[::-1] if not df_expenses.empty else df_expenses

正在進行 Google 授權...
✅ Google Sheets 連線成功！


In [2]:
# --- Gradio 互動函式 ---

def add_expense(amount: float, category: str, description: str):
    """
    紀錄一筆新的開銷，並儲存到 Google Sheet 中。
    """
    # 確保連線物件存在
    if 'worksheet' not in globals():
        return "❌ 錯誤：未成功連線到 Google Sheet，請先執行步驟一。", pd.DataFrame(columns=["Date", "Amount", "Category", "Description"])

    # 簡單驗證金額
    if amount <= 0:
        return "❌ 錯誤：金額必須大於 0。", view_expenses()[1]

    # 取得當前日期
    date = datetime.now().strftime("%Y-%m-%d")

    # 建立要新增的列資料 (必須與 Sheet 中的欄位順序一致)
    new_row = [date, amount, category, description]

    try:
        # 使用 gspread 的 append_row 方法新增資料
        worksheet.append_row(new_row, value_input_option='USER_ENTERED')

        # 刷新並取得最新的資料
        summary_text, updated_df = view_expenses()
        return f"✅ 成功紀錄: {category} - {amount} 元\n{summary_text}", updated_df

    except Exception as e:
        return f"❌ 儲存到 Google Sheets 失敗: {e}", view_expenses()[1]


def view_expenses():
    """
    從 Google Sheet 讀取所有開銷紀錄 (最新的在最上面)。
    """
    if 'worksheet' not in globals():
        return "❌ 錯誤：未成功連線到 Google Sheet，請先執行步驟一。", pd.DataFrame(columns=["Date", "Amount", "Category", "Description"])

    try:
        # 讀取所有帶有標頭的紀錄。⚠️ 您的試算表第一列必須是欄位名稱！
        data_records = worksheet.get_all_records()
        df = pd.DataFrame(data_records)

        if df.empty:
            return "目前沒有開銷紀錄。", pd.DataFrame(columns=["Date", "Amount", "Category", "Description"])

        # 計算總開銷
        if 'Amount' in df.columns:
            # 轉換為數字型態
            df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
            df.dropna(subset=['Amount'], inplace=True)
            total_spent = df["Amount"].sum()
        else:
            total_spent = 0.0

        # 返回總計資訊和反轉順序的 DataFrame (最新紀錄在最上)
        return f"總開銷：{total_spent:.2f} 元", df.iloc[::-1]

    except Exception as e:
        return f"❌ 讀取 Google Sheets 失敗: {e}", pd.DataFrame(columns=["Date", "Amount", "Category", "Description"])

# --- Gradio 介面設定 ---

# 預設的開銷類別選項
categories = [
    "食物/餐飲",
    "交通",
    "娛樂",
    "日常用品",
    "帳單/費用",
    "其他"
]

with gr.Blocks(title="Google Sheets 開銷紀錄器") as demo:
    gr.Markdown("# 🔗 Google Sheets 日常開銷紀錄器")

    # 紀錄開銷 Tab
    with gr.Tab("紀錄開銷"):
        with gr.Row():
            amount_input = gr.Number(label="金額 (元)", minimum=0.01)
            category_input = gr.Dropdown(categories, label="類別", value=categories[0])

        description_input = gr.Textbox(label="描述/備註", placeholder="例如：午餐便當、捷運車票")

        record_button = gr.Button("新增開銷", variant="primary")
        status_output = gr.Label(label="狀態")

    # 查看開銷 Tab
    with gr.Tab("查看所有開銷"):
        view_button = gr.Button("重新整理開銷紀錄", variant="secondary")

        summary_output = gr.Label(label="開銷總結")

        dataframe_output = gr.DataFrame(
            value=initial_df_display, # 初始顯示步驟一載入的資料
            headers=["日期", "金額", "類別", "描述"],
            wrap=True,
            interactive=False,
            label="所有開銷紀錄 (最新在最上)"
        )

    # --- 互動邏輯 ---
    record_button.click(
        fn=add_expense,
        inputs=[amount_input, category_input, description_input],
        outputs=[status_output, dataframe_output]
    )

    view_button.click(
        fn=view_expenses,
        inputs=[],
        outputs=[summary_output, dataframe_output]
    )

    # 介面初始載入時也執行一次查看
    demo.load(
        fn=view_expenses,
        outputs=[summary_output, dataframe_output]
    )

# 啟動 Gradio 介面 (share=True 會提供一個公開連結)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b29d3a039acc12f06b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
